<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [2]:
DATA_PATH = '/workspace/data/criteo/dac'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [4]:
df = df.sample(False, 0.5)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [7]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model'))

In [8]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1831172

In [9]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [10]:
!pip3.5 install hyperopt

## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [11]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [12]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [13]:
baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [14]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.7270969337250692

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [17]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

SPACE:                                                
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 6, 'eta': 0.5, 'min_child_weight': 50.0, 'featuresCol': 'features', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.9, 'objective': 'binary:logistic'}
LOG-LOSS: 0.5001253002549331, ROC-AUC: 0.7393513584890536
SPACE:                                                                           
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 6, 'eta': 0.1, 'min_child_weight': 50.0, 'featuresCol': 'features', 'num_round': 40, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.9, 'objective': 'binary:logistic'}
LOG-LOSS: 0.5080080104997449, ROC-AUC: 0.7293881814008916                        
SPACE:                                                                           
{'labelCol': 'label', 'p

In [18]:
best

{'eta': 0, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [19]:
num_round = num_round_choice[best['num_round']]
eta = eta_choice[best['eta']]

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

In [20]:
%%time

max_depth_choice  = [4, 5, 6, 7]
min_child_weight_choice = [10.0, 25.0, 50.0, 75.0, 100.0]

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
    
    # Fixed
    'num_round': num_round,
    'eta': eta,
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

SPACE:                                                
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 5, 'eta': 0.5, 'min_child_weight': 75.0, 'featuresCol': 'features', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.9, 'objective': 'binary:logistic'}
LOG-LOSS: 0.5008558412897175, ROC-AUC: 0.7383601163886406
SPACE:                                                                           
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 7, 'eta': 0.5, 'min_child_weight': 50.0, 'featuresCol': 'features', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.9, 'objective': 'binary:logistic'}
LOG-LOSS: 0.49986010789937063, ROC-AUC: 0.7395460197967275                       
SPACE:                                                                            
{'labelCol': 'label', 

In [21]:
best

{'max_depth': 3, 'min_child_weight': 2}

In [22]:
max_depth = max_depth_choice[best['max_depth']]
min_child_weight = min_child_weight_choice[best['min_child_weight']]

#### Tune gamma

In [23]:
%%time

gamma_choice  = [0.5, 0.8, 0.9, 0.95, 0.99]

space = {
    # Optimize
    'gamma': hp.choice('gamma', gamma_choice),

    # Fixed
    'num_round': num_round,
    'eta': eta,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': baseline_params['subsample'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5,
            trials=trials)

SPACE:                                               
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 7, 'eta': 0.5, 'min_child_weight': 50.0, 'featuresCol': 'features', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.5, 'objective': 'binary:logistic'}
LOG-LOSS: 0.49976966072181944, ROC-AUC: 0.7396315331238715
SPACE:                                                                           
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 7, 'eta': 0.5, 'min_child_weight': 50.0, 'featuresCol': 'features', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.8, 'objective': 'binary:logistic'}
LOG-LOSS: 0.49976595648479694, ROC-AUC: 0.739629262493551                        
SPACE:                                                                           
{'labelCol': 'label', '

In [24]:
best

{'gamma': 4}

In [25]:
gamma = gamma_choice[best['gamma']]

#### Tune subsample, colsample_bytree

In [26]:
%%time

subsample_choice = [0.5, 0.8, 0.9, 0.95]
colsample_bytree_choice = [0.5, 0.8, 0.9, 0.95]

space = {
    # Optimize
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    
    # Fixed
    'num_round': num_round,
    'eta': eta,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'gamma': gamma,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

SPACE:                                                
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 7, 'eta': 0.5, 'min_child_weight': 50.0, 'featuresCol': 'features', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.95, 'colsample_bytree': 0.8, 'gamma': 0.99, 'objective': 'binary:logistic'}
LOG-LOSS: 0.5000291152762375, ROC-AUC: 0.7391248129650421
SPACE:                                                                           
{'labelCol': 'label', 'predictionCol': 'prediction', 'eval_metric': 'logloss', 'max_depth': 7, 'eta': 0.5, 'min_child_weight': 50.0, 'featuresCol': 'features', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'nthread': 1, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.99, 'objective': 'binary:logistic'}
LOG-LOSS: 0.5000185902761987, ROC-AUC: 0.7393136701346823                        
SPACE:                                                                           
{'labelCol': 'label'

In [27]:
best

{'colsample_bytree': 2, 'subsample': 1}

In [28]:
subsample = subsample_choice[best['subsample']]
colsample_bytree = colsample_bytree_choice[best['colsample_bytree']]

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [34]:
%%time

reg_lambda_choice = [0.0, 1e-1, 1e-2, 1e-3]
alpha_choice = [0.0, 1e-1, 1e-2, 1e-3]

space = {
    # Optimize
    'reg_lambda': hp.choice('reg_lambda', reg_lambda_choice),
    'alpha': hp.choice('alpha', alpha_choice),
    
    # Fixed
    'num_round': num_round,
    'eta': eta,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)
best

SPACE:                                                
{'lambda': 0.1, 'predictionCol': 'prediction', 'subsample': 0.8, 'max_depth': 7, 'eta': 0.5, 'alpha': 0.1, 'gamma': 0.99, 'labelCol': 'label', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'featuresCol': 'features', 'min_child_weight': 50.0, 'nthread': 1}
LOG-LOSS: 0.49991685136862735, ROC-AUC: 0.7395248261655747
SPACE:                                                                            
{'lambda': 0.1, 'predictionCol': 'prediction', 'subsample': 0.8, 'max_depth': 7, 'eta': 0.5, 'alpha': 0.01, 'gamma': 0.99, 'labelCol': 'label', 'num_round': 100, 'silent': 0, 'nworkers': 1, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'featuresCol': 'features', 'min_child_weight': 50.0, 'nthread': 1}
LOG-LOSS: 0.49995706504042214, ROC-AUC: 0.7394848780717508                        
SPACE:                                   

{'alpha': 3, 'reg_lambda': 2}

In [35]:
reg_lambda = reg_lambda_choice[best['reg_lambda']]
alpha = alpha_choice[best['alpha']]

### Lower the learning rate and decide the optimal parameters

In [47]:
tuned_xgb_params = {    
    'num_round': num_round,
    'eta': eta,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    
    'reg_lambda': reg_lambda,
    'alpha': alpha,
    
    **static_params
}
tuned_xgb_params

{'alpha': 0.001,
 'colsample_bytree': 0.9,
 'eta': 0.5,
 'eval_metric': 'logloss',
 'featuresCol': 'features',
 'gamma': 0.99,
 'labelCol': 'label',
 'max_depth': 7,
 'min_child_weight': 50.0,
 'nthread': 1,
 'num_round': 100,
 'nworkers': 1,
 'objective': 'binary:logistic',
 'predictionCol': 'prediction',
 'reg_lambda': 0.01,
 'silent': 0,
 'subsample': 0.8}

In [48]:
# Lowering the learning rate, i.e. eta
tuned_xgb_params['eta'] /= 10
tuned_xgb_params

{'alpha': 0.001,
 'colsample_bytree': 0.9,
 'eta': 0.05,
 'eval_metric': 'logloss',
 'featuresCol': 'features',
 'gamma': 0.99,
 'labelCol': 'label',
 'max_depth': 7,
 'min_child_weight': 50.0,
 'nthread': 1,
 'num_round': 100,
 'nworkers': 1,
 'objective': 'binary:logistic',
 'predictionCol': 'prediction',
 'reg_lambda': 0.01,
 'silent': 0,
 'subsample': 0.8}

In [49]:
xgb_tuned = XGBoostEstimator(**tuned_xgb_params).fit(train_df)

In [50]:
log_loss = logloss(xgb_tuned, val_df, probabilities_col='probabilities')
roc_auc = rocauc(xgb_tuned, val_df, probabilities_col='probabilities')
log_loss, roc_auc

(0.5053375826043808, 0.7329226393107793)

## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [88]:
from pyspark.ml.classification import LogisticRegression

def log_reg_objective(space):
    logistic_regression = LogisticRegression(**space)
    print('SPACE:', space)
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = logistic_regression.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [89]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'probabilityCol': "probabilities",
    'rawPredictionCol': "rawPrediction",
    'family': 'binomial',
}

In [90]:
baseline_params = {
    'maxIter': 100,
    'tol': 1e-6,
    'aggregationDepth': 2,

    'standardization': True,
    'fitIntercept': True,
    'threshold': 0.5,

    'regParam': 0.001,
    'elasticNetParam': 0.1,
}

baseline_log_reg_model = LogisticRegression(**{**static_params, **baseline_params}).fit(train_df)

In [91]:
log_loss = logloss(baseline_log_reg_model, val_df, probabilities_col='probabilities')
roc_auc = rocauc(baseline_log_reg_model, val_df, probabilities_col='probabilities')
log_loss, roc_auc

(0.527406253894106, 0.7029238695402533)

### Tune the training regime parameters (maxIter, tol, aggregationDepth)

In [92]:
%%time

max_iter_choice = [50, 100, 150]
tol_choice = [1e-4, 1e-5, 1e-6, 1e-7]
aggregation_depth_choice = [2, 3]

space = {
    # Optimize
    'maxIter': hp.choice('maxIter', max_iter_choice),
    'tol': hp.choice('tol', tol_choice),
    'aggregationDepth': hp.choice('aggregationDepth', aggregation_depth_choice),
    
    # Fixed
    'standardization': baseline_params['standardization'],
    'fitIntercept': baseline_params['fitIntercept'],
    'threshold': baseline_params['threshold'],

    'regParam': baseline_params['regParam'],
    'elasticNetParam': baseline_params['elasticNetParam'],
    
    **static_params
}

trials = Trials()
best = fmin(fn=log_reg_objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

best

SPACE:                                                
{'threshold': 0.5, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 3, 'tol': 0.0001, 'fitIntercept': True, 'maxIter': 100, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LOG-LOSS: 0.52740317429959, ROC-AUC: 0.7029338903756053
SPACE:                                                                        
{'threshold': 0.5, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LOG-LOSS: 0.5274008770364704, ROC-AUC: 0.7029273379344602                     
SPACE:                                                       

LOG-LOSS: 0.5274066977935213, ROC-AUC: 0.7029227724203663                        
SPACE:                                                                           
{'threshold': 0.5, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 3, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 150, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LOG-LOSS: 0.5274066977935213, ROC-AUC: 0.7029227724203682                        
SPACE:                                                                           
{'threshold': 0.5, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 3, 'tol': 1e-06, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LOG

{'aggregationDepth': 0, 'maxIter': 0, 'tol': 1}

In [93]:
max_iter = max_iter_choice[best['maxIter']]
tol = tol_choice[best['tol']]
aggregation_depth = aggregation_depth_choice[best['aggregationDepth']]

### Tune the internals of the model (standardization, fitIntercept, threshold)

In [94]:
%%time

standardization_choice = [True, False]
fit_intercept_choice = [True, False]
threshold_choice = [0.3, 0.4, 0.5, 0.6, 0.7]

space = {
    # Optimize
    'standardization': hp.choice('standardization', standardization_choice),
    'fitIntercept': hp.choice('fitIntercept', fit_intercept_choice),
    'threshold': hp.choice('threshold', threshold_choice),
    
    # Fixed
    'maxIter': max_iter,
    'tol': tol,
    'aggregationDepth': aggregation_depth,

    'regParam': baseline_params['regParam'],
    'elasticNetParam': baseline_params['elasticNetParam'],
    
    **static_params
}

trials = Trials()
best = fmin(fn=log_reg_objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

best

SPACE:                                                
{'threshold': 0.5, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': False, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LOG-LOSS: 0.532695208180498, ROC-AUC: 0.6985678818649214
SPACE:                                                                         
{'threshold': 0.6, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LOG-LOSS: 0.5274008770364704, ROC-AUC: 0.702927337934458                       
SPACE:                                                     

LOG-LOSS: 0.5274748891683849, ROC-AUC: 0.7025790546625578                        
SPACE:                                                                           
{'threshold': 0.4, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': False, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LOG-LOSS: 0.532695208180498, ROC-AUC: 0.6985678818649215                         
SPACE:                                                                           
{'threshold': 0.6, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.1, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': False, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.001, 'family': 'binomial'}
LO

{'fitIntercept': 0, 'standardization': 0, 'threshold': 3}

In [95]:
standardization = standardization_choice[best['standardization']]
fit_intercept = fit_intercept_choice[best['fitIntercept']]
threshold = threshold_choice[best['threshold']]

### Tune the regularization params (regParam, elasticNetParam)

In [113]:
%%time

reg_param_choice = [0.0, 1e-1, 1e-2, 1e-3]
elastic_net_param_choice = [0.0, 1e-1, 1e-2, 1e-3, 1e-4]

space = {
    # Optimize
    'regParam': hp.choice('regParam', reg_param_choice),
    'elasticNetParam': hp.choice('elasticNetParam', elastic_net_param_choice),
    
    # Fixed
    'maxIter': max_iter,
    'tol': tol,
    'aggregationDepth': aggregation_depth,

    'standardization': standardization,
    'fitIntercept': fit_intercept,
    'threshold': threshold,
    
    **static_params
}

trials = Trials()
best = fmin(fn=log_reg_objective,
            space=space,
            algo=tpe.suggest,
            max_evals=25,
            trials=trials)

best

SPACE:                                                
{'threshold': 0.6, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.001, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.01, 'family': 'binomial'}
LOG-LOSS: 0.5283168620914481, ROC-AUC: 0.7013060028294131
SPACE:                                                                          
{'threshold': 0.6, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.001, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.1, 'family': 'binomial'}
LOG-LOSS: 0.5340970699983068, ROC-AUC: 0.6994847445821358                       
SPACE:                                                  

LOG-LOSS: 0.5284330975562895, ROC-AUC: 0.7024076594179998                        
SPACE:                                                                           
{'threshold': 0.6, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.01, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.1, 'family': 'binomial'}
LOG-LOSS: 0.535580895148847, ROC-AUC: 0.7002052118718166                         
SPACE:                                                                           
{'threshold': 0.6, 'labelCol': 'label', 'predictionCol': 'prediction', 'standardization': True, 'elasticNetParam': 0.0, 'rawPredictionCol': 'rawPrediction', 'featuresCol': 'features', 'aggregationDepth': 2, 'tol': 1e-05, 'fitIntercept': True, 'maxIter': 50, 'probabilityCol': 'probabilities', 'regParam': 0.01, 'family': 'binomial'}
LOG-LO

{'elasticNetParam': 1, 'regParam': 3}

In [114]:
reg_param = reg_param_choice[best['regParam']]
elastic_net_param = elastic_net_param_choice[best['elasticNetParam']]

### Decide on the optimal parameters

In [115]:
tuned_log_reg_params = {
    'maxIter': max_iter,
    'tol': tol,
    'aggregationDepth': aggregation_depth,

    'standardization': standardization,
    'fitIntercept': fit_intercept,
    'threshold': threshold,
    
    'regParam': reg_param,
    'elasticNetParam': elastic_net_param,

    **static_params
}
tuned_log_reg_params

{'aggregationDepth': 2,
 'elasticNetParam': 0.1,
 'family': 'binomial',
 'featuresCol': 'features',
 'fitIntercept': True,
 'labelCol': 'label',
 'maxIter': 50,
 'predictionCol': 'prediction',
 'probabilityCol': 'probabilities',
 'rawPredictionCol': 'rawPrediction',
 'regParam': 0.001,
 'standardization': True,
 'threshold': 0.6,
 'tol': 1e-05}

In [116]:
tuned_log_reg_model = LogisticRegression(**tuned_log_reg_params).fit(train_df)

In [117]:
log_loss = logloss(tuned_log_reg_model, val_df, probabilities_col='probabilities')
roc_auc = rocauc(tuned_log_reg_model, val_df, probabilities_col='probabilities')
log_loss, roc_auc

(0.5274008770364704, 0.7029273379344584)

---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [118]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте сравнение на отложенной выборке

In [119]:
all_metrics = {}
CONTROL_NAME = 'xgb baseline'
for model_name, model in [
    (CONTROL_NAME, baseline_model),
    ('xgb tuned', xgb_tuned),
    ('log_reg baseline', baseline_log_reg_model),
    ('log_reg tuned', tuned_log_reg_model),
]:
    model_test_metrics = {
        'logloss': logloss(model, test_df, probabilities_col='probabilities'),
        'rocauc': rocauc(model, test_df, probabilities_col='probabilities')
    }
    all_metrics[model_name] = model_test_metrics

Итоговая таблица

In [120]:
get_ate(all_metrics, CONTROL_NAME)

,log_reg baseline ate %,log_reg tuned ate %,metric,xgb tuned ate %
0,-2.998783,-2.997869,rocauc,0.872427
1,2.908179,2.907683,logloss,-0.990472
